# Music and Mental Health Analysis

This notebook analyzes survey data to explore relationships between music listening behaviors and self-reported mental health outcomes.


## Setup
- Put the dataset at `../data/mxmh_survey_results.csv`
- Install dependencies: `pip install -r ../requirements.txt`


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, levene, mannwhitneyu
import plotly.express as px


In [ ]:
DATA_PATH = "../data/mxmh_survey_results.csv"
df = pd.read_csv(DATA_PATH)

# Basic validations
assert len(df) > 0, "Dataset failed to load — it has 0 rows."
for col in ["Anxiety", "Depression", "Hours per day", "Fav genre"]:
    assert col in df.columns, f"Missing required column: {col}"

df.head()


## Genre Categorization (Calm vs High Energy)


In [ ]:
calm_genres = ["Classical", "Folk", "Lofi", "Jazz", "Country"]
high_energy_genres = ["Rock", "EDM", "Hip hop", "Metal", "Pop", "Rap"]

def genre_type(genre: str) -> str:
    if genre in calm_genres:
        return "Calm"
    if genre in high_energy_genres:
        return "High Energy"
    return "Other"

df["Genre Type"] = df["Fav genre"].apply(genre_type)
df[["Fav genre", "Genre Type"]].head()


## Example Visualization (Plotly)


In [ ]:
genre_summary = df.groupby("Genre Type")[["Anxiety", "Depression"]].mean().reset_index()
fig = px.bar(
    genre_summary,
    x="Genre Type",
    y=["Anxiety", "Depression"],
    barmode="group",
    title="Average Anxiety and Depression by Genre Type",
    labels={"value": "Average Score", "variable": "Metric"},
)
fig.show()


## Statistical Testing (Mann–Whitney U)
If normality is violated, Mann–Whitney U provides a non-parametric alternative.


In [ ]:
calm_anx = df[df["Genre Type"] == "Calm"]["Anxiety"].dropna()
high_anx = df[df["Genre Type"] == "High Energy"]["Anxiety"].dropna()

print("Shapiro (Calm):", shapiro(calm_anx))
print("Shapiro (High Energy):", shapiro(high_anx))
print("Levene:", levene(calm_anx, high_anx))

u_anx, p_anx = mannwhitneyu(calm_anx, high_anx, alternative="less")
print("Mann–Whitney U (Anxiety):", u_anx, p_anx)
